# Get the data   
Note: automate as much as possible so you can easily get fresh data.

- [x] List the data you need and how much you need.  

> * FMI API weather forecast data from https://github.com/kipe/fmi
> * FMI historical data downloaded with a csv from https://en.ilmatieteenlaitos.fi/download-observations
> * Helen historical data downloaded with a csv from https://www.helen.fi/en/solar-panels/solar-power-plants/suvilahti-solar-power-plant
> * Calculation of theoretical solar irradiance: 
 - http://sgemfinalreport.fi/files/WP611_photovoltaics%20HP%20Hellman.pdf
 - https://hal.archives-ouvertes.fr/hal-00361373/document

- [x] Find and document where you can get that data.
- [x] Check how much space it will take.
>Negligible amount
- [x] Check legal obligations, and get the authorization if necessary.
>* fmi_weather license: __MIT__
>* GHI article citations
>* helen data and weather observation data are freely available
- [x] Get access authorizations.
>Wasn't necessary
- [x] Create a workspace (with enough storage space).
- [x] Get the data.
>Downloading data 1/4/2015 - 30/4/2020 in hourly resolution
>Weather data from Helsinki, Kumpula - the closest observation station to Suvilahti
- [x] Convert the data to a format you can easily manipulate (without changing the data itself).
- [x] Ensure sensitive information is deleted or protected (e.g., anonymized).
- [x] Check the size and type of data (time series, sample, geographical, etc.).
>Timeseries converted into UTC
- [x] Sample a test set, put it aside, and never look at it (no data snooping!).
>Do that


Let's sample a test set and set it aside. To do that, we need to first read all the data together and save them for later use in a more comfortable format for feature engineering and model building.

In [63]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pytz

def multiple_years_files_gather_data(year_range_start,year_range_end, string):
    df = pd.DataFrame()
    #adding to one dataframe
    for year in range(year_range_start,year_range_end):
        year_start = str(year)
        year_end = str(year+1)
        df_temp = pd.read_csv('../datasets/'+ string + year_start + '_'+ year_end + '.csv')
        df = pd.concat([df,df_temp],axis=0)
    return df

#### Weather observation data

In [30]:
year_range_start = 15
year_range_end = 20
df_weather = multiple_years_files_gather_data(year_range_start=year_range_start, year_range_end=year_range_end, string='initial_weather_obs_')

In [31]:
df_weather.tail()

,Year,m,d,Time,Time zone,Cloud amount (1/8),Pressure (msl) (hPa),Relative humidity (%),Precipitation intensity (mm/h),Snow depth (cm),Air temperature (degC),Dew-point temperature (degC),Horizontal visibility (m),Wind direction (deg),Gust speed (m/s),Wind speed (m/s)
8779,2020,4,30,20:00,UTC,0.0,1015.5,66.0,0.0,0.0,3.2,-2.5,50000.0,89.0,6.1,4.4
8780,2020,4,30,21:00,UTC,7.0,1015.6,67.0,0.0,0.0,3.2,-2.3,50000.0,87.0,5.6,3.4
8781,2020,4,30,22:00,UTC,7.0,1015.3,74.0,0.0,0.0,3.4,-0.9,50000.0,90.0,8.4,4.9
8782,2020,4,30,23:00,UTC,0.0,1015.3,73.0,0.0,0.0,3.2,-1.3,50000.0,94.0,7.5,3.8
8783,2020,5,1,00:00,UTC,0.0,1015.4,73.0,0.0,0.0,3.2,-1.3,50000.0,102.0,8.1,4.8


In [57]:
#creating a datetime index for concatenation
df_weather['datetime'] = df_weather['Year'].astype(str)+'-'+df_weather['m'].astype(str)+'-'+df_weather['d'].astype(str)+' '+ df_weather['Time'].astype(str)
df_weather['datetime_converted'] = pd.to_datetime(df_weather['datetime'], utc=True)
df_weather = df_weather.set_index(pd.DatetimeIndex(df_weather['datetime_converted'],dayfirst=True))

In [58]:
df_weather

,Year,m,d,Time,Time zone,Cloud amount (1/8),Pressure (msl) (hPa),Relative humidity (%),Precipitation intensity (mm/h),Snow depth (cm),Air temperature (degC),Dew-point temperature (degC),Horizontal visibility (m),Wind direction (deg),Gust speed (m/s),Wind speed (m/s),datetime,datetime_converted
datetime_converted,,,,,,,,,,,,,,,,,,
2015-04-01 00:00:00+00:00,2015,4,1,00:00,UTC,7.0,990.6,100.0,0.0,0.0,0.5,0.5,9690.0,107.0,2.7,2.3,2015-4-1 00:00,2015-04-01 00:00:00+00:00
2015-04-01 01:00:00+00:00,2015,4,1,01:00,UTC,5.0,991.0,100.0,0.0,0.0,0.4,0.4,5690.0,109.0,3.7,2.7,2015-4-1 01:00,2015-04-01 01:00:00+00:00
2015-04-01 02:00:00+00:00,2015,4,1,02:00,UTC,8.0,990.9,100.0,0.0,0.0,0.9,0.9,7680.0,118.0,2.9,2.3,2015-4-1 02:00,2015-04-01 02:00:00+00:00
2015-04-01 03:00:00+00:00,2015,4,1,03:00,UTC,7.0,990.8,100.0,0.0,0.0,1.1,1.1,6110.0,33.0,3.3,2.7,2015-4-1 03:00,2015-04-01 03:00:00+00:00
2015-04-01 04:00:00+00:00,2015,4,1,04:00,UTC,7.0,991.0,100.0,0.0,0.0,0.7,0.7,2800.0,28.0,2.9,2.1,2015-4-1 04:00,2015-04-01 04:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-30 20:00:00+00:00,2020,4,30,20:00,UTC,0.0,1015.5,66.0,0.0,0.0,3.2,-2.5,50000.0,89.0,6.1,4.4,2020-4-30 20:00,2020-04-30 20:00:00+00:00
2020-04-30 21:00:00+00:00,2020,4,30,21:00,UTC,7.0,1015.6,67.0,0.0,0.0,3.2,-2.3,50000.0,87.0,5.6,3.4,2020-4-30 21:00,2020-04-30 21:00:00+00:00
2020-04-30 22:00:00+00:00,2020,4,30,22:00,UTC,7.0,1015.3,74.0,0.0,0.0,3.4,-0.9,50000.0,90.0,8.4,4.9,2020-4-30 22:00,2020-04-30 22:00:00+00:00


#### Global radiation data

In [32]:
df_rad = multiple_years_files_gather_data(year_range_start=year_range_start, year_range_end=year_range_end, string='initial_global_rad_')

In [34]:
df_rad.tail()

,Year,m,d,Time,Time zone,Global radiation (W/m2)
8780,2020,4,30,20:00,UTC,-2.3
8781,2020,4,30,21:00,UTC,-1.8
8782,2020,4,30,22:00,UTC,-2.0
8783,2020,4,30,23:00,UTC,-2.5
8784,2020,5,1,00:00,UTC,-2.1


In [59]:
#creating a datetime index for concatenation
df_rad['datetime'] = df_rad['Year'].astype(str)+'-'+df_rad['m'].astype(str)+'-'+df_rad['d'].astype(str)+' '+ df_rad['Time'].astype(str)
df_rad['datetime_converted'] = pd.to_datetime(df_rad['datetime'], utc=True)
df_rad = df_rad.set_index(pd.DatetimeIndex(df_rad['datetime_converted'],dayfirst=True))

In [61]:
df_rad.head()

,Year,m,d,Time,Time zone,Global radiation (W/m2),datetime,datetime_converted
datetime_converted,,,,,,,,
2015-04-01 00:00:00+00:00,2015,4,1,00:00,UTC,-0.3,2015-4-1 00:00,2015-04-01 00:00:00+00:00
2015-04-01 01:00:00+00:00,2015,4,1,01:00,UTC,-1.1,2015-4-1 01:00,2015-04-01 01:00:00+00:00
2015-04-01 02:00:00+00:00,2015,4,1,02:00,UTC,-0.4,2015-4-1 02:00,2015-04-01 02:00:00+00:00
2015-04-01 03:00:00+00:00,2015,4,1,03:00,UTC,-0.8,2015-4-1 03:00,2015-04-01 03:00:00+00:00
2015-04-01 04:00:00+00:00,2015,4,1,04:00,UTC,4.6,2015-4-1 04:00,2015-04-01 04:00:00+00:00


#### PV data

In [35]:
df_solar = pd.read_csv('../datasets/initial_energy_data.csv', delimiter=';')

In [37]:
df_solar.tail()

,Date,Value (kWh)
44581,2015-04-01T04:00:00,0
44582,2015-04-01T03:00:00,0
44583,2015-04-01T02:00:00,0
44584,2015-04-01T01:00:00,0
44585,2015-04-01T00:00:00,0


In [64]:
#creating a datetime index for concatenation
df_solar['datetime_converted'] = pd.to_datetime(df_solar['Date'], utc=False)
df_solar['datetime_converted']=df_solar['datetime_converted'].dt.tz_localize(pytz.timezone('Europe/Helsinki'),nonexistent='NaT',ambiguous='NaT').dt.tz_convert('UTC')
df_solar = df_solar.set_index(pd.DatetimeIndex(df_solar['datetime_converted'],dayfirst=True))

In [65]:
df_solar

,Date,Value (kWh),datetime_converted
datetime_converted,,,
2020-05-01 14:00:00+00:00,2020-05-01T17:00:00,37,2020-05-01 14:00:00+00:00
2020-05-01 13:00:00+00:00,2020-05-01T16:00:00,112,2020-05-01 13:00:00+00:00
2020-05-01 12:00:00+00:00,2020-05-01T15:00:00,193,2020-05-01 12:00:00+00:00
2020-05-01 11:00:00+00:00,2020-05-01T14:00:00,162,2020-05-01 11:00:00+00:00
2020-05-01 10:00:00+00:00,2020-05-01T13:00:00,220,2020-05-01 10:00:00+00:00
...,...,...,...
2015-04-01 01:00:00+00:00,2015-04-01T04:00:00,0,2015-04-01 01:00:00+00:00
2015-04-01 00:00:00+00:00,2015-04-01T03:00:00,0,2015-04-01 00:00:00+00:00
2015-03-31 23:00:00+00:00,2015-04-01T02:00:00,0,2015-03-31 23:00:00+00:00


## Notice:
Because I have made a mistake (perhaps) and sneak peeked at the data. I know that there is no temperature available from Kumpula for almost a year. There are several options how to deal with it:

1. Remove these values completely. Then, we lose almost 8760 values, approximately
2. Impute those values from somewhere else. For example from the data from a nearby station with another two options
 - Completely pasting them (copy paste, quite irresponsible, but given that there is a about 3km from station to station
 - Creating some sort of model for imputation, which requires more data
 
Therefore, I have predownloaded the Kaisaniemi (the next closest station) Air temperature and Dew point temperature for 2018-2019. We see later what happens.

In [67]:
# before concatenating, need to remove the duplicate rows in each dataframe
df_weather = df_weather[~df_weather.index.duplicated(keep='first')]
df_rad = df_rad[~df_rad.index.duplicated(keep='first')]
df_solar = df_solar[~df_solar.index.duplicated(keep='first')]

Let's put together the dataframes and sample ourselves a Train and a Test set.

In [68]:
frames = [df_weather, df_solar, df_rad]
df = pd.concat([df_weather,df_solar, df_rad], axis=1)

In [73]:
df.isnull().mean()

Year                              0.000404
m                                 0.000404
d                                 0.000404
Time                              0.000404
Time zone                         0.000404
Cloud amount (1/8)                0.001929
Pressure (msl) (hPa)              0.001458
Relative humidity (%)             0.001458
Precipitation intensity (mm/h)    0.003588
Snow depth (cm)                   0.015520
Air temperature (degC)            0.118061
Dew-point temperature (degC)      0.037948
Horizontal visibility (m)         0.022024
Wind direction (deg)              0.006145
Gust speed (m/s)                  0.005966
Wind speed (m/s)                  0.006145
datetime                          0.000404
datetime_converted                0.000404
Date                              0.000224
Value (kWh)                       0.000224
datetime_converted                0.000247
Year                              0.000404
m                                 0.000404
d          

In [74]:
len(df_weather)

44569

In [75]:
len(df)

44587

In [76]:
len(df_solar)

44577

In [77]:
len(df_rad)

44569